In [1]:
import pandas as pd
import numpy as np

In [4]:
import pandas as pd
import numpy as np
reviewsDF = pd.read_csv('DatasetFinal.csv')
reviewsDF['Brand Name'] = reviewsDF['Brand Name'].str.lower()
reviews = reviewsDF.Reviews.tolist()
brands = reviewsDF['Brand Name'].tolist()
ratings = reviewsDF['Rating'].tolist()
print("Loaded " + str(len(reviews)) + " reviews.")

Loaded 59255 reviews.


In [5]:
reviewsDF.head()

,Product Name,Brand Name,Rating,Reviews,Reviews_Votes
0,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,1,Received a used phone . Very disappointed!!,2
1,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,4,Update 8/31/20 IF YOU DIDN'T RECEIVE YOUR WIRE...,188
2,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,5,Great phone and food quality. I was able to us...,0
3,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,5,"La pantalla se ve impresionante, se escucha me...",30
4,Samsung Electronics Galaxy Note 20 Ultra 5G Fa...,samsung,1,"It came today. When opened the box, there was ...",1


In [6]:

import nltk
def get_all_phrases_containing_tar_wrd(target_word, tar_passage, left_margin = 5, right_margin = 5):
    """
        Function to get all the pharses that contain the target word in a text/passage tar_passage.
        Workaround to save the output given by nltk Concordance function
        
        str target_word --> aspect to be searched for
        str tar_passage  --> sentence extracted from a customer review
        int left_margin int right_margin --> left_margin and right_margin allocate the number of words/punctuation before and after target word
        Left margin will take note of the beginning of the text
    """
     
   
    tokens = nltk.word_tokenize(tar_passage)
    tokens = [x for x in tokens if len(x)>2] 
 
    text = nltk.Text(tokens)
 
 
    c = nltk.ConcordanceIndex(text.tokens, key = lambda s: s.lower())
 

    
    concordance_txt = ([text.tokens[  list(map(lambda x: x-5 if (x-left_margin)>0 else 0,[offset]))[0]:offset+right_margin] for offset in c.offsets(target_word)])

 
    return [''.join([x+' ' for x in con_sub]) for con_sub in concordance_txt]

In [7]:
def cleanText(raw_text, remove_stopwords=False, stemming=False, split_text=False):
    '''
    Convert a raw review to a cleaned review
    '''
    text = BeautifulSoup(raw_text, 'lxml').get_text()  #remove html
    letters_only = re.sub("[^a-zA-Z]", " ", text)  # remove non-character
    words = letters_only.lower().split() # convert to lower case 
    
    if remove_stopwords: # remove stopword
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
        
    if stemming==True: # stemming
#         stemmer = PorterStemmer()
        stemmer = SnowballStemmer('english') 
        words = [stemmer.stem(w) for w in words]
        
    if split_text==True:  # split text
        return (words)
    
    return( " ".join(words))

In [8]:
from nltk import sent_tokenize


relevant_aspects = ['battery', 'screen', 'camera', 'performance']


phrases = []
brand = []
rating = []
aspects = []
review_num = []
counter = 0

for review in reviews:
    try:
       
        for sentence in sent_tokenize(review):
            for  important_word in relevant_aspects:
               
                phrases_in_sentence = get_all_phrases_containing_tar_wrd(important_word, sentence, left_margin = 5, right_margin = 5) 
                for phrase in phrases_in_sentence:
                    brand.append(brands[counter])
                    rating.append(ratings[counter])
                    aspect = 'other'
                    for imp_word in relevant_aspects:
                        if imp_word in phrase.lower():
                            aspect = imp_word
                    aspects.append(aspect)
                    phrases.append(phrase)
                    review_num.append(counter)
    except:
        pass
    counter = counter + 1

print('Extracted ' + str(len(phrases)) + ' phrases from the reviews')


phrasesDF = pd.DataFrame()
phrasesDF['brand'] = brand 
phrasesDF['phrase'] = phrases
phrasesDF['rating'] = rating
phrasesDF['aspect'] = aspects
phrasesDF['review_id'] = review_num
phrasesDF.head()

Extracted 22131 phrases from the reviews


,brand,phrase,rating,aspect,review_id
0,samsung,even the note but this screen here bests all t...,4,screen,1
1,samsung,all this makes for outstanding screen,4,screen,1
2,samsung,Great main camera,4,camera,1
3,samsung,result but all all the camera top notch all its,4,camera,1
4,samsung,find phone with better main camera the market,4,camera,1


In [9]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
vader_sentiment = []  
vader_sentiment_neg = [] 
vader_sentiment_pos = []
sid = SentimentIntensityAnalyzer()
for sentence in phrasesDF['phrase']:
    try:
        vader_sentiment.append(sid.polarity_scores(sentence)['compound'])
        vader_sentiment_pos.append(sid.polarity_scores(sentence)['pos'])
        vader_sentiment_neg.append(sid.polarity_scores(sentence)['neg'])
    except:
        vader_sentiment.append(0.5)
phrasesDF['vader_sentiment'] = pd.Series(vader_sentiment)   
phrasesDF['vader_sentiment_neg'] = pd.Series(vader_sentiment_neg)   
phrasesDF['vader_sentiment_pos'] = pd.Series(vader_sentiment_pos)

In [10]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,vader_sentiment,vader_sentiment_neg,vader_sentiment_pos
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.0000,0.0,0.000
1,samsung,all this makes for outstanding screen,4,screen,1,0.6124,0.0,0.444
2,samsung,Great main camera,4,camera,1,0.6249,0.0,0.672
3,samsung,result but all all the camera top notch all its,4,camera,1,0.2960,0.0,0.196
4,samsung,find phone with better main camera the market,4,camera,1,0.4404,0.0,0.293


In [11]:
phrasesDF['comp_score'] = phrasesDF['vader_sentiment'].apply(lambda c: 'best' if c >=0.75 else ('good' if c >=0.5 else ('better' if c >=0.25 else ('ok' if c >=0 else ('bad' if c <=-0.25 else ('very bad' if c <=-0.5 else ('worst' if c <=-0.75 else 'very worst')))))))

In [12]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,vader_sentiment,vader_sentiment_neg,vader_sentiment_pos,comp_score
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.0000,0.0,0.000,ok
1,samsung,all this makes for outstanding screen,4,screen,1,0.6124,0.0,0.444,good
2,samsung,Great main camera,4,camera,1,0.6249,0.0,0.672,good
3,samsung,result but all all the camera top notch all its,4,camera,1,0.2960,0.0,0.196,better
4,samsung,find phone with better main camera the market,4,camera,1,0.4404,0.0,0.293,better


In [13]:
phrasesDF['label'] = phrasesDF['comp_score'].apply(lambda c: 4 if c =='best' else (3 if c =='good' else (2 if c =='better' else (1 if c =='ok' else (-1 if c =='bad' else (-2 if c =='very bad' else (-3 if c =='worst' else -4)))))))

In [14]:
phrasesDF.head()

,brand,phrase,rating,aspect,review_id,vader_sentiment,vader_sentiment_neg,vader_sentiment_pos,comp_score,label
0,samsung,even the note but this screen here bests all t...,4,screen,1,0.0000,0.0,0.000,ok,1
1,samsung,all this makes for outstanding screen,4,screen,1,0.6124,0.0,0.444,good,3
2,samsung,Great main camera,4,camera,1,0.6249,0.0,0.672,good,3
3,samsung,result but all all the camera top notch all its,4,camera,1,0.2960,0.0,0.196,better,2
4,samsung,find phone with better main camera the market,4,camera,1,0.4404,0.0,0.293,better,2


In [15]:
def modelEvaluation(predictions, y_test_set):
    #Print model evaluation to predicted result 
    
    print ("\nAccuracy on validation set: {:.4f}".format(accuracy_score(y_test_set, predictions)))
    #print "\nAUC score : {:.4f}".format(roc_auc_score(y_test_set, predictions))
    print ("\nClassification report : \n", metrics.classification_report(y_test_set, predictions))
    print ("\nConfusion Matrix : \n", metrics.confusion_matrix(y_test_set, predictions))

NAIVE BAYES

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup  
import re
import nltk
from nltk.corpus import stopwords 

In [20]:
#split data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(phrasesDF['phrase'], phrasesDF['label'], test_size=0.3, random_state=0)
    
    
#If the label are words instead of numbers, they can be replaced programmatically using following two lines
#     y_train = label_binarize(y_train, classes=[0, 1, 2])
#     y_test = label_binarize(y_test, classes=[0, 1, 2])

In [21]:
# Preprocess text data in training set and validation set
x_train_cleaned = []
x_test_cleaned = []

for d in x_train:
    x_train_cleaned.append(cleanText(d))

for d in x_test:
    x_test_cleaned.append(cleanText(d)) 

In [25]:
# Fit and transform the training data to a document-term matrix using CountVectorizer
countVect = CountVectorizer() 
x_train_countVect = countVect.fit_transform(x_train_cleaned)
print ("Number of features : %d \n" %len(countVect.get_feature_names())) #6378 
print ("Show some feature names : \n", countVect.get_feature_names()[::1000])

Number of features : 4768 

Show some feature names : 
 ['abd', 'def', 'impressive', 'plastic', 'stayed']


In [26]:
# Train MultinomialNB classifier
mnb = MultinomialNB()
mnb.fit(x_train_countVect, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
# Evaluate the model on validaton set
predictions = mnb.predict(countVect.transform(x_test_cleaned))
modelEvaluation(predictions, y_test)


Accuracy on validation set: 0.7354

Classification report : 
               precision    recall  f1-score   support

          -4       0.83      0.21      0.33       232
          -1       0.82      0.71      0.76       935
           1       0.82      0.89      0.85      2900
           2       0.63      0.58      0.60      1021
           3       0.56      0.74      0.64      1070
           4       0.74      0.42      0.54       482

    accuracy                           0.74      6640
   macro avg       0.73      0.59      0.62      6640
weighted avg       0.74      0.74      0.73      6640


Confusion Matrix : 
 [[  48   38  109   21   16    0]
 [   1  662  181   50   41    0]
 [   6   58 2582  117  126   11]
 [   2   31  197  594  183   14]
 [   1   13   72  143  793   48]
 [   0    1    2   22  253  204]]


In [31]:
print ("Accuracy of this Naivebays = " + str(metrics.accuracy_score(y_test, predictions)))

Accuracy of this Naivebays = 0.7353915662650602


In [30]:
print(mnb.predict(countVect.transform([
    "It was bad"
])))

[-1]
